In [3]:
weather_df = pd.read_csv("AIFrienz S3_v2\\weather_v2.csv")
weather_df = weather_df[weather_df["station"] == "ASOS"]

In [4]:
weather_df.head()

,area,datetime,temp,temp_QCFlag,prec,prec_QCFlag,ws,ws_QCFlag,wd,wd_QCFlag,...,vis,sfctype,weathernumber,sfctemp,sfctemp_QCFlag,landtemp_5cm,landtemp_10cm,landtemp_20cm,landtemp_30cm,station
0,184,2018-02-01 01:00,4.7,0.0,NaN,NaN,3.6,0.0,20.0,0.0,...,1950.0,NaN,NaN,2.9,0.0,3.7,3.9,4.2,4.9,ASOS
1,184,2018-02-01 02:00,4.8,0.0,NaN,NaN,2.6,0.0,360.0,0.0,...,1865.0,NaN,NaN,2.9,0.0,3.6,3.8,4.2,4.9,ASOS
2,184,2018-02-01 03:00,4.8,0.0,NaN,NaN,4.6,0.0,20.0,0.0,...,1855.0,NaN,NaN,3.0,0.0,3.6,3.8,4.3,4.9,ASOS
3,184,2018-02-01 04:00,4.5,0.0,NaN,NaN,5.7,0.0,20.0,0.0,...,1425.0,NaN,NaN,3.0,0.0,3.6,3.8,4.2,5.0,ASOS
4,184,2018-02-01 05:00,4.5,0.0,NaN,9.0,4.4,0.0,20.0,0.0,...,1043.0,NaN,1.0,3.0,0.0,3.5,3.8,4.2,4.9,ASOS


In [5]:
weather_df['datetime'] = pd.to_datetime(weather_df['datetime'])

In [6]:
weather_df = weather_df[["datetime", "area", "temp", "prec", "ws", "humid"]]

In [7]:
jeju_df = weather_df[weather_df["area"] == 184].reset_index(drop = True)
gosan_df = weather_df[weather_df["area"] == 185].reset_index(drop = True)
sungsan_df = weather_df[weather_df["area"] == 188].reset_index(drop = True)
seogwipo_df = weather_df[weather_df["area"] == 189].reset_index(drop = True)

In [8]:
# 고산, 서귀포는 결측값 확인해야함
len(jeju_df), len(gosan_df), len(sungsan_df), len(seogwipo_df)

(20110, 20104, 20110, 20108)

In [9]:
# null 값 처리
jeju_df["prec"] = jeju_df["prec"].fillna(0)
jeju_df = jeju_df.interpolate(method='values')

In [10]:
jeju_df

,datetime,area,temp,prec,ws,humid
0,2018-02-01 01:00:00,184,4.7,0.0,3.6,58.0
1,2018-02-01 02:00:00,184,4.8,0.0,2.6,57.0
2,2018-02-01 03:00:00,184,4.8,0.0,4.6,62.0
3,2018-02-01 04:00:00,184,4.5,0.0,5.7,62.0
4,2018-02-01 05:00:00,184,4.5,0.0,4.4,62.0
...,...,...,...,...,...,...
20105,2020-05-18 19:00:00,184,17.8,0.0,5.5,93.0
20106,2020-05-18 20:00:00,184,16.3,0.4,4.2,93.0
20107,2020-05-18 21:00:00,184,16.6,0.0,4.3,85.0
20108,2020-05-18 22:00:00,184,16.6,0.0,4.3,67.0


#### houly_smp 데이터
- datatime 포멧중 자정은 당일 24:00시로 표기되어 있음
    - 당일 24:00시 -> 익일 00:00으로 데이터 변환
- 이후 weather_df와 병합

In [12]:
smp_df = pd.read_csv("AIFrienz S3_v2\\hourly_smp_v2.csv")

In [13]:
smp_df["date"] = smp_df["datetime"].str.split(' ').str[0]
smp_df["time"] = smp_df["datetime"].str.split(' ').str[1]

In [14]:
smp_df["time"] = smp_df["time"].mask(smp_df["time"] == "24:00", "00:00")
smp_df["date"] = pd.to_datetime(smp_df['date'])
smp_df.loc[smp_df["time"] == "00:00", "date"] += np.timedelta64(1,'D')

In [15]:
smp_df["date"] = smp_df["date"].astype(str)
smp_df["time"] = smp_df["time"].astype(str)

In [16]:
smp_df["datetime"] = smp_df["date"].str.cat(smp_df["time"], sep=' ')
smp_df['datetime'] = pd.to_datetime(smp_df['datetime'])

In [17]:
smp_df = smp_df.drop(columns=['date', 'time'])
smp_df

,datetime,smp
0,2018-02-01 01:00:00,116.84
1,2018-02-01 02:00:00,116.84
2,2018-02-01 03:00:00,116.84
3,2018-02-01 04:00:00,116.84
4,2018-02-01 05:00:00,116.84
...,...,...
20107,2020-05-18 20:00:00,91.02
20108,2020-05-18 21:00:00,111.05
20109,2020-05-18 22:00:00,111.73
20110,2020-05-18 23:00:00,112.75
